In [ ]:
%cd ..

In [ ]:
import torch
import math
import matplotlib.pyplot as plt
import sys
import os

from PIL import Image
from diffusers import FluxPipeline
from torch import Tensor
from torchvision import transforms
from IPython.display import clear_output

DTYPE = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=DTYPE)
pipe.to(device)
print(f"Loaded pipeline to device {device}")

In [ ]:
from rectified_flow.models.flux_dev import FluxWrapper
from rectified_flow.rectified_flow import RectifiedFlow
from rectified_flow.utils import set_seed

set_seed(0)

height = 1025
width = 1025

flux_model = FluxWrapper(
    pipeline=pipe,
    height=height,
    width=width,
    dtype=DTYPE,
    device=device,
	seed=0,
)

rectflow = RectifiedFlow(
    data_shape=flux_model.packed_latent_shape,
    model=flux_model,
    interp="straight",
    device=device,
	dtype=DTYPE,
)

In [ ]:
# Sample and store sampling info for several following samplers
X_0 = rectflow.sample_source_distribution(batch_size=1)
time_grid = flux_model.prepare_time_grid(num_steps=50)
print(f"X_0: {X_0.shape}")
print(f"time_grid: {time_grid}")	

def print_time_callback(sampler): # demo callback function, e.g. one-step prediction
    """A callback function to print the current time t, refreshing the Jupyter Notebook output."""
    clear_output(wait=True)
    print(f"Current time: {sampler.t:.4f}")
    
my_callback = [print_time_callback]

In [ ]:
from rectified_flow.samplers import rf_samplers_dict

for sampler_name, sampler_class in rf_samplers_dict.items():
	print(f"Sampler: {sampler_name}, class: {sampler_class}")

In [ ]:
from rectified_flow.samplers import EulerSampler

euler_sampler = EulerSampler(
    rectified_flow=rectflow,
    callbacks=my_callback,
)

euler_sampler.sample_loop(
    X_0=X_0,
    time_grid=time_grid,
    prompt="A photo of a cat holding a camera",
    guidance_scale=3.5,
)

X_1 = euler_sampler.trajectories[-1]
print(X_1.shape)

img = flux_model.unpack_and_decode(X_1)
plt.imshow(img)

In [ ]:
noise_refresh_sampler = rf_samplers_dict["noise_refresh"](
    rectified_flow=rectflow,
    callbacks=my_callback,
)

noise_refresh_sampler.sample_loop(
    X_0=X_0,
    time_grid=time_grid,
    prompt="A photo of a cat holding a camera",
    guidance_scale=3.5,
)

X_1 = noise_refresh_sampler.trajectories[-1]
print(X_1.shape)

img = flux_model.unpack_and_decode(X_1)
plt.imshow(img)

In [ ]:
overshoot_sampler = rf_samplers_dict["overshooting"](
    rectified_flow=rectflow,
    callbacks=my_callback,
)

overshoot_sampler.sample_loop(
    X_0=X_0,
    time_grid=time_grid,
    prompt="A photo of a cat holding a camera",
    guidance_scale=3.5,
)

X_1 = overshoot_sampler.trajectories[-1]
print(X_1.shape)

img = flux_model.unpack_and_decode(X_1)
plt.imshow(img)